In [1]:
import torch
import random
import math

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, _ = tokenizer.encode('how are you', max_length=6)

input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

input_ids, attention_mask, tokenizer.decode(input_ids)

/root/miniconda3/envs/cuda117/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(tensor([   1,    1,    0, 9178,   32,   47]),
 tensor([0, 0, 1, 1, 1, 1]),
 '<pad><pad><s>how are you')

In [2]:
from transformers import AutoModelForCausalLM

model_actor = AutoModelForCausalLM.from_pretrained('model/rlhf',
                                                   torch_dtype=torch.float16,
                                                   device_map='cuda')

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.60s/it]


In [3]:
import json

with open('dataset/eval.json') as f:
    lines = f.readlines()

lines = random.sample(lines, k=8)
lines = [json.loads(i) for i in lines]

len(lines), lines[0]

(8,
 {'prompt': 'Human: context= CREATE TABLE table_name_86 (rd VARCHAR, _time VARCHAR, date VARCHAR) question= What was the Rd. Time for October 3, 2009? Assistant:',
  'chosen': 'SELECT rd, _time FROM table_name_86 WHERE date = "october 3, 2009"',
  'rejected': '',
  'response': 'SELECT rd, _time FROM table_name_86 WHERE date = "october 3, 2009"'})

In [4]:
for data in lines:
    input_ids, _ = tokenizer.encode(data['prompt'], max_length=256)
    input_ids, attention_mask = tokenizer.pad_to_left(input_ids)

    input_ids = input_ids.unsqueeze(0).to('cuda')
    attention_mask = attention_mask.unsqueeze(0).to('cuda')

    generate = model_actor.generate(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    max_length=512,
                                    pad_token_id=tokenizer.pad_token_id,
                                    eos_token_id=tokenizer.eos_token_id)

    generate = generate[0, 256:].to('cpu')

    print(tokenizer.decode(generate))
    print(data['chosen'])
    print('===============')

select RD FROM table_name_86 WHERE _time = "october 3, 2009" AND date = "october 3, 2009"</s>
SELECT rd, _time FROM table_name_86 WHERE date = "october 3, 2009"
select ENGINE from TABLE_NAME_27 where YEAR = "61" and POINTS = "0" and CHASSIS = "porsche 718"</s>
SELECT engine FROM table_name_27 WHERE points = 0 AND chassis = "porsche 718" AND year = 1961
select STATE from TABLE_NAME_43 where TEAM = "MUMBAI"</s>
SELECT state FROM table_name_43 WHERE team = "mumbai"
select MIN(2nd_baseman) from TABLE_12142298_2 where FIRST_BASEMAN = "nomar gARCIAAPARRA"</s>
SELECT second_baseman FROM table_12142298_2 WHERE first_baseman = "Nomar Garciaparra"
select SCIENCE_AGENCY from TABLE_NAME_17 where NAME = "HERSChel Space Observatory"</s>
SELECT space_agency FROM table_name_17 WHERE name = "herschel space observatory"
select DATE from TABLE_NAME_14 where PEARCE = "Samba"</s>
SELECT pair FROM table_name_14 WHERE date = "october 14, 2008" AND dance = "samba"
select POINTS from TABLE_NAME_96 where PLAYED